<h1><center>Identifying general genomic feature trade-offs and their phylogenetic relationships among soil resident microorganisms</center></h1>

<center>Alicia McElwee</center>
<center>BIOCB 6840</cen

The goal of this project is to understand how general genomic traits may help inform our understanding of microbes' ecological role in biogeochemical cycling, specifically carbon cycling. I am using sequenced and annotated genomes from the RefSoil database of soil resident microorganisms to understand what general genomic features show tradeoff characteristics that may be indicative of larger ecological tradeoffs the organism has made which influence its ecological functioning. I also want to test how phylogenetically linked these tradeoff traits are, i.e. could these tradeoff traits tell us more about an organism's ecologically functioning then their phylogenetic positioning?
This notebook specifically contains the code used to download the genome sequences and annotations from NCBI, calculate the general genomic features, create the phylogenetic tree using PhyloPhlAn 3.0, and attempt to test phylogenetic signal using BayesTraits. The tradeoff feature analysis were done in separate applications (R and Matlab) so have been uploaded separately but are dicussed briefly here.

<h2>Downloading Genome and Genome Annotation Files</h2>

I must first download onto my lab's server the genome sequence and genome annoatation files from NCBI where the files for the 928 RefSoil organisms are stored. For some reason the people who created the RefSoil database only give nucleotide accession and species denotation in their excel file about the database, not the assembly accession number. I also realized that many of the species names had been updated since the RefSoil list was compiled in 2015. I wrote a script to use the NCBI Entrez tool to first use the nucleotide accession number to find the updated organism name which I entered into the spreadsheet which lists all the original given data about the RefSoil entries. I then used the updated organism name to search for the assembly accession number. Finally, I used the assembly accession number to download the RefSeq genome sequence and genome annotation files from NCBI. It was stored in a folder titled Plan_B as I had originally tried downloading the database using the given organism names but realized many of those were not correct and had to redownload the entire dataset.

In [ ]:
# Getting updated organism name from nucleotide accession number included in RefSoil database

from subprocess import Popen, PIPE, STDOUT
import os
import json
import csv

os.system("export PATH=${HOME}/edirect:${PATH}")
print(os.getcwd())

with open('database_info_RefSoil_organism.tsv','r') as tsvin:
    
    for line in csv.reader(tsvin, delimiter='\t'):
        
        if line[0] == 'RefSoil ID':
            pass
        else:
            nucl_accession_line = line[1]
            nucl_access_list = nucl_accession_line.split(',')
            nucl_accession = nucl_access_list[0]
            
            nucl_acces_search_cmd_list = ['/home/alicia/edirect/esearch -db nucleotide -query', nucl_accession, '| /home/alicia/edirect/efetch -format docsum | grep -h "<Organism>"']
            nucl_acces_search_cmd = ' '.join(nucl_acces_search_cmd_list)
            #print(nucl_acces_search_cmd)
            os.system("export PATH=${HOME}/edirect:${PATH}")
            os.system(nucl_acces_search_cmd)


In [ ]:
## Getting RefSeq Accession numbers for RefSoil database entries from the new organism names

from subprocess import Popen, PIPE, STDOUT
import os
import json
import csv

os.environ["PATH"] += os.pathsep + os.pathsep.join(["/home/alicia/anaconda3/envs/ncbi_datasets/bin/"])
print (os.environ["PATH"])


with open('Plan_B/NEW_RefSoil_organism_database_info.tsv','r') as tsvin:
    
    for line in csv.reader(tsvin, delimiter='\t'):
        
        if line[0] == 'RefSoil ID':
            pass
        else:
            ID = line[0]
            org_name_list = ["'", line[18],"'"]
            org_name = ''.join(org_name_list)
            cmd_list = ["datasets summary genome taxon", org_name, "--annotated"] 
            cmd = ' '.join(cmd_list)
            
            p = Popen(cmd, shell=True, stdout=PIPE,  close_fds=True)
            output = p.stdout.read()
          
            try:
                output_2 = json.loads(output)
                
                if 'reports' in output_2:
                    print(ID)
                    print(output_2['reports'][0]['accession'])
                else:
                    print(ID)
                    print("*Sad NO Buzzer noises*") # These are to let me know that the program could not find the assembly accession using the organism name
                    
            except:
                print(ID)
                print("*Sad NO Buzzer noises*") # These are to let me know that the program could not find the assembly accession using the organism name

After I did this and added the assembly accession number to the excel sheet for the entries of the RefSoil database, I had to go back and manually curate the database entries to spot check that the assembly accession numbers match the organism entry in the RefSoil database and properly marking those which had been repressed so should not be included in the analysis as, I have found, their files do not download correctly and they lead to problems later on. Of the 928 entries originally included in RefSoil I had to get rid of 38, leaving me with 890 entries used in this project. Below I use the assembly accession numbers to download the RefSoil entries:

In [ ]:
## Downloading genome sequence and gff files for each entry using their NEW assembly accession number

import os
import csv

os.environ["PATH"] += os.pathsep + os.pathsep.join(["/home/alicia/anaconda3/envs/ncbi_datasets/bin/"])
print (os.environ["PATH"])


with open('Plan_B/NEW_RefSoil_organism_database_info.tsv','r') as tsvin:
    for line in csv.reader(tsvin, delimiter='\t'):
        refsoil_ID = line[0]
        file_name_list = ["Plan_B/New_RefSoil_Backup/", refsoil_ID, ".zip"]
        file_name = ''.join(file_name_list)
        
        if line[0] == 'RefSoil ID':
            pass
        
        elif line[20] == 'n':
            pass
        
        elif line[20] == 'y':
            accession = line[19]
            
            if accession == 0 or accession == None:
                pass
            
            else:
                print("Starting: ", refsoil_ID)
                dwnld_genome_data = ["datasets download genome accession", accession, "--include gff3,genome,seq-report --annotated --assembly-source 'RefSeq' --filename", file_name, '>/dev/null 2>&1']
                dwnld_genome_cmd = ' '.join(dwnld_genome_data)
                os.system(dwnld_genome_cmd)
                print("Finished: ", refsoil_ID)

<h2>Calculating General Genome Feature Values</h2>

Using the genome .fna file and the annotation .gff file to determine values for the general genome features of interest. I am using one block of code to do this so I can iterate through each RefSoil entry and unzip the files only once to access the genome and genome annotation files instead of having to unzip them multiple times. For this analysis I used all 890 RefSoil entries I had downloaded.

In [ ]:
import gffutils
from Bio import SeqIO
from pprint import pprint
import os
import csv
import glob

# Setting up keywords to use for counting each general feature of interest
meme_trans_keys = ["transmembrane transporter","symporter","antiporter"]
transcript_keys = ["transcriptional regulator","transcriptional repressor","transcription factor","operon repressor","regulation of transcription","transcription regulation","transcription initiation","transcription termination"]
mac_keys = ["methyl-accepting chemotaxis protein"]
antibiotic_keys = ["antibiotic biosynthesis","antibiotic response", "response to antibiotic", "antibiotic catabolic","antibiotic"]

# Making csv file to store calculated genome feature values in
with open('Plan_B/NEW_Genome_feature_values.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    header = ["RefSoil_ID", "CDS_count", "Coding Density", "rRNA_count","Membrane_trans_count","Membrane_trans_density","TranscriptionFactor_Count","TranscriptionFactor_density", "MAC_count","MAC_density", "AntiBioticRelated_Count","AntiBioticRelated_Density","G/C Content", "Genome_Size"]
    writer.writerow(header)
    
    # Iterating through Truncated RefSoil entries
    with open('Plan_B/NEW_RefSoil_organism_database_info.tsv','r') as tsvin:

        for line in csv.reader(tsvin, delimiter='\t'):
        
            if line[0] == 'RefSoil ID':
                pass
        
            elif line[20] == 'n': # Using list for truncated RefSoil or either y or n
                pass
            
            elif line[20] == 'y':
                
                # Collecting needed info
                refsoil_ID = line[0]
                refsoil_ID_loc_list = ['Plan_B/', refsoil_ID]
                refsoil_ID_loc = ''.join(refsoil_ID_loc_list)
                accession = line[19]
                
                # Uncompressing RefSoil files for the entry
                zip_file_name_list = ["Plan_B/New_RefSoil/",refsoil_ID, ".zip"]
                zip_file_name = ''.join(zip_file_name_list)
            
                unzip_cmd_list = ['unzip -d', refsoil_ID_loc, zip_file_name]
                unzip_cmd = ' '.join(unzip_cmd_list)
                os.system(unzip_cmd)
                
                # Finding genome sequence file path
                genome_file_loc_list = [refsoil_ID_loc, "/ncbi_dataset/data/", accession, "/G*"]
                genome_file_loc = ''.join(genome_file_loc_list)
                genome_file_loc = ''.join(glob.glob(genome_file_loc))
                
                # Finding gff file path
                gff_file_loc_list = [refsoil_ID_loc, "/ncbi_dataset/data/", accession, "/genomic.gff"]
                gff_file_loc = ''.join(gff_file_loc_list)

                # Creating gff database with gffutils to search through for features of interest
                db = gffutils.create_db(gff_file_loc, "Plan_B/gff.db", merge_strategy = "create_unique", keep_order = True)
                
                # Setting Count values back to 0 for next entry
                rRNA_count = 0
                CDS_count = 0
                memeTrans_count = 0
                transcript_count = 0
                mac_count = 0
                antiBiotic_count = 0

                # Iterating through all genome features
                for feature in db.all_features():

                    # Counting number of copies of rRNA based on number of 16S genes
                    if feature.attributes['gbkey']==['rRNA']:
                        if '16S' in feature.attributes['product'][0]:
                            rRNA_count +=1

                    if feature.attributes['gbkey'] == ['CDS']:

                        # Counting Number of coding sequences
                        CDS_count +=1

                        try:
                            go_product = ''.join(feature.attributes['product'])

                            # Counting Methly-accepting chemotaxis genes
                            if any(x in go_product for x in mac_keys):
                                mac_count +=1 

                            # Counting Some of the antiobiotic related genes
                            if any(x in go_product for x in antibiotic_keys):
                                antiBiotic_count +=1 

                            else:

                                try: 
                                    go_function = ''.join(feature.attributes['go_function'])
                                    go_process = ''.join(feature.attributes['go_process'])

                                    # Counting Membrane Transporter Genes
                                    if any(x in go_function for x in meme_trans_keys) or any(x in go_process for x in meme_trans_keys) or any(x in go_product for x in meme_trans_keys):
                                        memeTrans_count +=1      

                                    # Counting Transcription Factor Genes
                                    if any(x in go_function for x in transcript_keys) or any(x in go_process for x in transcript_keys) or any(x in go_product for x in transcript_keys):
                                        transcript_count +=1

                                    # Counting Rest of antibiotic related genes
                                    if any(x in go_function for x in antibiotic_keys) or any(x in go_process for x in antibiotic_keys):
                                        antiBiotic_count +=1

                                except:
                                    pass

                        except:
                            pass


                # Finding Genome Length
                print(genome_file_loc)
                f = open(genome_file_loc, "r")
                seq_len = 0
                G_C = 0

                for line in f:
                    if line.startswith('>'):
                        pass
                    else:
                        line_bp_count = len(line)
                        seq_len = seq_len + line_bp_count
                        for bp in line:
                            if bp == 'g' or bp == 'G' or bp == 'c' or bp == 'C':
                                G_C += 1
                    
                # Calculating other values of interest
                G_C_content = G_C/seq_len
                coding_density = CDS_count/seq_len
                memeTrans_density = memeTrans_count/CDS_count
                transcript_density = transcript_count/CDS_count
                mac_density = mac_count/CDS_count
                antiBiotic_density = antiBiotic_count/CDS_count
                

                new_row = [refsoil_ID, CDS_count, coding_density, rRNA_count, memeTrans_count, memeTrans_density, transcript_count, transcript_density, mac_count, mac_density, antiBiotic_count, antiBiotic_density, G_C_content, seq_len]
                writer.writerow(new_row)
                
                # Deleting current gff.db file
                os.system("rm Plan_B/gff.db")
                
                # Deleting uncompressed file
                cmd_rm_list = ["rm -r", refsoil_ID_loc]
                cmd_rm = ' '.join(cmd_rm_list)
                os.system(cmd_rm)

The code above produced the New_Genome_feature_values.csv output with all 13 general genomic feature values calculated for all 890 RefSoil entries.

<h2>Analysis of Genome Features for Tradeoff Relationships</h2>

<h3>Analysis with ParTI</h3>

For the ParTI analysis, I wrote the new_genome_feature_analysis.m script located in the Tradeoff_Feature_Analysis/ParTI folder based on the example scripts that came with the program. The inputs to this program included: values_only_LOG_NewGenomeNewFeaturevalues.csv which contains the log transformed feature values with the header column and the first row identifying which RefSoil entry the data is from removed (as is required by ParTI) and feature_names.list which is a list of the genome feature names to try to input the into the continuous feature parameters to do enrichment analysis of the features in the different identified archetypes. The outputs produced by this program are in Tradeoff_Feature_Analysis/ParTI/Output

<h3>Backup R PCA Analysis</h3>

Since I was unable to get the feature enrichment analysis to work in ParTI, I did a simple PCA analysis in R. The R markdown file for this (Backup_R_PCA_Analysis.Rmd) is in Tradeoff_Feature_Analysis/R and the input file is LOGTranform_NewGenomeNewFeaturevalues.xlsx. Note that this file contains the exact same log transformed feature values as was used in the ParTI analysis but retains the entry identification column and the row with the column headers. The 2D PCA loading plot is found in Tradeoff_Feature_Analysis/R/2D_Feature_PCAloadingPlot.png. I could not save the 3D loading plot as it requires the rgl package to view.

<h2>Phylogenetic Tree Creation with PhyloPhlAn 3.0 </h2>

I used PhyloPhlAn 3.0 (Asnicar et al. 2020. Nature Communications, 11(1), Article 1) to construct a phylogenetic tree from the genome sequences of all 890 entries using 400 marker genes. I ran this program through the command line on my labs server in an anaconda environment but used this notebook to the preprocessing steps. I followed the instructions for setting parameters and running the program's' supertree method found at https://github.com/biobakery/phylophlan/wiki. I realized quickly that creating a multi-locus tree would too long for the time span of this project, so I decided to truncate my dataset from 890 to ~200 entries using a weighted random yes or no generator:

In [ ]:
## Generating Weighted Random y or n for 887 entries to select around ~200 entries for further analysis

import csv
import random

with open('database_info_RefSoil_organism.tsv','r') as tsvin:
    
    for line in csv.reader(tsvin, delimiter='\t'):
        
        if line[0] == 'RefSoil ID':
            pass
        elif line[18] == 'n': # If entry was already not part of the 887, then put n again for it
            print('n') 
        elif line[18] == 'y': # If entry was a part of the 887, then take weighted random choice of n or y
            rand_value = ''.join(random.choices(['n','y'], weights = (78,22)))
            print(rand_value)

I then copied these weighted random y's and n's into the Excel file about the database entries and used it to denote which entries to delete from my current working to clean it up directory before proceeding with phylogenetic tree creation. There were 203 entries which were selected to use for downstream phylogenetic analysis.

In [ ]:
## Using Random Weighted y and n values to truncate RefSoil entries for this project (deleting them from working directory)

import os
import csv
import random

print(os.getcwd())


with open('Plan_B/NEW_RefSoil_organism_database_info.tsv','r') as tsvin:
    
    for line in csv.reader(tsvin, delimiter='\t'):
        
        if line[0] == 'RefSoil ID':
            pass
        
        elif line[21] == 'y':
            pass
        
        elif line[21] == 'n':
            ID = line[0]
            zip_name_list = ['Plan_B/New_RefSoil/', ID, '.zip']
            zip_name = ''.join(zip_name_list)
            
            rm_cmd_list = ['rm', zip_name]
            rm_cmd = ' '.join(rm_cmd_list)
            os.system(rm_cmd)

I next had to copy all the genome sequence files from the 203 entries for phylogenetic analysis into a single folder and convert them to a zip format to save space on the server.

In [ ]:
## Collecting genome files of Truncated RefSoil into a single folder for PhyloPhlAn 3.0

import os
import csv

print(os.getcwd())


with open('Plan_B/NEW_RefSoil_organism_database_info.tsv','r') as tsvin:

    for line in csv.reader(tsvin, delimiter='\t'):
        
        if line[0] == 'RefSoil ID':
            pass
        
        elif line[21] == 'n': # Using list for truncated RefSoil or either y or n
            pass
            
        elif line[21] == 'y':
            
            # Getting needed info
            refsoil_ID = line[0]
            refsoil_file_list =['Plan_B/New_RefSoil/', refsoil_ID]
            refsoil_file = ''.join(refsoil_file_list)
            
            accession = line[19]
            print(refsoil_ID)
            zip_file_name_list = ['Plan_B/New_RefSoil/', refsoil_ID, ".zip"]
            zip_file_name = ''.join(zip_file_name_list)
            
            
            # Unzipping File
            unzip_cmd_list = ['unzip -d', refsoil_file, zip_file_name]
            unzip_cmd = ' '.join(unzip_cmd_list)
            os.system(unzip_cmd)
            
            # Compressing genome sequence file
            genome_file_loc_list = ['Plan_B/New_RefSoil/', refsoil_ID, "/ncbi_dataset/data/", accession, "/G*"]
            genome_file_loc = ''.join(genome_file_loc_list)
            cmpress_cmd_list = ["gzip", genome_file_loc]
            cmpress_cmd = ' '.join(cmpress_cmd_list)
            os.system(cmpress_cmd)
            
            # Renaming genome sequence file
            New_compressed_loc_list = ['Plan_B/New_RefSoil/', refsoil_ID, "/ncbi_dataset/data/", accession,"/", refsoil_ID, '.fna.gz']
            New_compressed_loc = ''.join(New_compressed_loc_list)
            Old_compressed_loc_list = ['Plan_B/New_RefSoil/', refsoil_ID, "/ncbi_dataset/data/", accession, "/G*"]
            Old_compressed_loc = ''.join(Old_compressed_loc_list)
            cmd_rename_list = ["mv", Old_compressed_loc, New_compressed_loc]
            cmd_rename = ' '.join(cmd_rename_list)
            os.system(cmd_rename)
            
            # Moving compressed sequence file to genome_files folder
            destination_name_list = ["/home/alicia/class/Comp_gene/Plan_B/New_genome_files/", refsoil_ID, '.fna.gz']
            destination_name = ''.join(destination_name_list)
            cmd_mv_list = ["mv", New_compressed_loc, destination_name]
            cmd_mv = ' '.join(cmd_mv_list)
            print(cmd_mv)
            os.system(cmd_mv)
            
            # Deleting uncompressed file
            cmd_rm_list = ["rm -r", refsoil_file]
            cmd_rm = ' '.join(cmd_rm_list)
            os.system(cmd_rm)
            

I was now ready to run PhyloPhlAn 3.0 on my server, which I did using the command line code below. New_genome_files is the name of the directory where the genome sequence files for the 203 RefSoil entries were copied to. Note that supertree_aa.cfg is a default configuration file made by PhyloPhlAn where I only had to update the area asking where ASTRAL was located. phylophlan.tsv is a list of which substitution model to use for each marker gene when analyzing the multi-sequence alignments to create the gene trees.

<code>phylophlan -i New_genome_files -d phylophlan --diversity high -f supertree_aa.cfg --force_nucleotides --maas phylophlan.tsv  --accurate --nproc 50</code>

This produced the New_genome_files.tre output containing the phylogenetic tree in Newick format.

<h2>Testing Phylogenetic Signal of Features</h2>

This program was run on the command line of both my lab's server and my personal computer. In both cases, the inputs were phylophlan_accurate_with_NewRefSoil_genome_features.nex (the phylogenetic tree which I had to convert from Newick to Nexus format using the Newick2Nexus tool (https://github.com/josephwb/Newick2Nexus) since BayesTraits only uses Nexus trees) and LOGTranform_NEW_GenomeNEWFeaturevalues.tsv (the log transformed feature values with the column headers removed as is required for BayesTraits). These input files can be found in the BayesTraits_Input folder. The command use to run the program was:   

<code>BayesTraitsV4 phylophlan_accurate_with_NewRefSoil_genome_features.nex LOGTranform_NEWGenomeNEWFeaturevalues.tsv</code>

But when I ran this program on both the lab server and my personal computer, it would not then prompt me to select an analysis method. It would just produce a bunch of numbers and then exit the program. I tried to solve this but did ran out of time with this project o figure out why that was happening.